In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from joblib import Parallel, delayed


def fill_missing_values_with_regression(df):
    df['time'] = pd.to_datetime(df['time'])
    df['year'] = df['time'].dt.year
    df['week'] = df['time'].dt.isocalendar().week
    df['day_of_week'] = df['time'].dt.dayofweek
    df['hour'] = df['time'].dt.hour
    df['minute'] = df['time'].dt.minute

    df['hour_sin'] = np.sin(2 * np.pi * df['time'].dt.hour / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['time'].dt.hour / 24)
    df['day_of_week_sin'] = np.sin(2 * np.pi * df['time'].dt.dayofweek / 7)
    df['day_of_week_cos'] = np.cos(2 * np.pi * df['time'].dt.dayofweek / 7)
    df['week_sin'] = np.sin(2 * np.pi * df['time'].dt.isocalendar().week / 52)
    df['week_cos'] = np.cos(2 * np.pi * df['time'].dt.isocalendar().week / 52)

    # Добавление сезонного шума для каждого дня недели и каждого часа дня
    np.random.seed(42)  # Фиксируем seed для воспроизводимости
    df['day_of_week_noise'] = np.random.normal(0, 100, len(df))  # Сезонный шум для дня недели
    df['hour_noise'] = np.random.normal(0, 50, len(df))  # Сезонный шум для часа дня

    def fill_missing_value(idx, row, df):
        start_idx = idx
        end_idx = idx
        while start_idx > 0 and pd.isna(df.loc[start_idx - 1, 'P_l']):
            start_idx -= 1
        while end_idx < len(df) - 1 and pd.isna(df.loc[end_idx + 1, 'P_l']):
            end_idx += 1

        previous_years_data = df[(df['time'].dt.month == row['time'].month) & (df['time'].dt.day == row['time'].day)]

        X_before = df.loc[start_idx:idx - 1,
                   ['hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos', 'day_of_week_noise', 'hour_noise']]
        y_before = df.loc[start_idx:idx - 1, 'P_l']
        X_after = df.loc[idx + 1:end_idx,
                  ['hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos', 'day_of_week_noise', 'hour_noise']]
        y_after = df.loc[idx + 1:end_idx, 'P_l']
        X_previous_years = previous_years_data[
            ['hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos', 'day_of_week_noise', 'hour_noise']]
        y_previous_years = previous_years_data['P_l']

        X_train = pd.concat([X_before, X_after, X_previous_years])
        y_train = pd.concat([y_before, y_after, y_previous_years])
        print("X_train length:", len(X_train))
        print("y_train length:", len(y_train))
        X_train.interpolate(method='linear', inplace=True, limit_direction='both')

        # Интерполяция только для NaN значений в y_train
        y_train.interpolate(method='linear', inplace=True, limit_direction='both')

        model = LinearRegression()
        model.fit(X_train, y_train)


        X_missing = np.array([
            row['hour_sin'],
            row['hour_cos'],
            row['day_of_week_sin'],
            row['day_of_week_cos'],
            row['day_of_week_noise'],
            row['hour_noise']
        ]).reshape(1, -1)
        predicted_value = model.predict(X_missing)[0]

        return idx, predicted_value
    missing_data = df[df['P_l'].isnull()]
    results = Parallel(n_jobs=-1)(
        delayed(fill_missing_value)(idx, row, df.copy()) for idx, row in missing_data.iterrows())

    for idx, predicted_value in results:
        df.loc[idx, 'P_l'] = predicted_value

    return df


df_origin = pd.read_csv('filled_P_l.csv')
df = fill_missing_values_with_regression(df_origin)


df['time'] = pd.to_datetime(df['time'])

plt.figure(figsize=(10, 6))
plt.plot(df['time'], df['P_l'])
plt.title('Time Series Plot (Matplotlib)')
plt.xlabel('Time')
plt.ylabel('Value')
plt.grid(True)
plt.show()

X_train length: 2304
y_train length: 2304
X_train length: 2304
y_train length: 2304
X_train length:X_train length: 2304
y_train length: 2304
 2304
y_train length: 2304
X_train length:X_train length: 2304 
2304
y_train length: y_train length: 2304
2304
X_train length: 2304
y_train length: 2304
X_train length: 2304
y_train length: 2304


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not ha

X_train length: 2375
y_train length: 2375


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


X_train length: 2375
y_train length: 2375
X_train length: 2375
y_train length: 2375


...

X_train length: 4574
y_train length: 4574
X_train length: 4574
y_train length: 4574


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


X_train length: 4574
y_train length: 4574


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


PicklingError: Could not pickle the task to send it to the workers.